In [1]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import pingouin as pg
import seaborn as sns
from biopsykit.questionnaires.utils import compute_scores, wide_to_long
from biopsykit.utils.dataframe_handling import convert_nan
from fau_colors import cmaps, register_fausans_font

from stressgait_analysis.dataset import StressGaitDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
deploy_type = "local"

config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
gait_data_path = Path(config_dict[deploy_type]["gait_data_path"])

base_path


PosixPath('/home/rzlin/ys64ofuj/Data/StressGait/Data')

In [3]:
dataset = StressGaitDataset(base_path, coarse_condition=True, gait_data_path=gait_data_path, specify_bouts=True)
dataset.create_index()

,participant,condition,bout
0,VP_01,omc,1
1,VP_01,omc,2
2,VP_02,control,1
3,VP_02,control,2
4,VP_05,control,1
...,...,...,...
77,VP_45,omc,2
78,VP_46,control,1
79,VP_46,control,2
80,VP_47,omc,1


In [4]:
dataset[0].load_force_plate_data()

,GRF_x_1,GRF_y_1,GRF_z_1,COP_x_1,COP_y_1,COP_z_1,M_x_1,M_y_1,M_z_1,GRF_x_2,GRF_y_2,GRF_z_2,COP_x_2,COP_y_2,COP_z_2,M_x_2,M_y_2,M_z_2
time,,,,,,,,,,,,,,,,,,
0.000,-1.795417e+01,2.360873e+02,-7.648138e+00,0.128431,0.0,1.522868e-01,-3.593265e-06,6.524470e+00,4.721882e-06,17.990130,339.609496,9.949480,-0.082576,0.0,0.196193,-0.000009,1.253937,-4.229413e-06
0.001,-1.795193e+01,2.360025e+02,-7.597609e+00,0.128418,0.0,1.521526e-01,-3.507014e-06,6.535135e+00,4.561056e-06,17.942118,339.812500,9.954998,-0.082582,0.0,0.196001,-0.000008,1.276605,-4.110929e-06
0.002,-1.794973e+01,2.359177e+02,-7.545980e+00,0.128405,0.0,1.520182e-01,-3.421817e-06,6.545469e+00,4.402011e-06,17.894158,340.016852,9.960754,-0.082587,0.0,0.195811,-0.000008,1.299830,-3.993124e-06
0.003,-1.794754e+01,2.358331e+02,-7.493465e+00,0.128390,0.0,1.518839e-01,-3.337865e-06,6.555469e+00,4.245189e-06,17.846363,340.222112,9.966797,-0.082591,0.0,0.195624,-0.000008,1.323488,-3.876405e-06
0.004,-1.794536e+01,2.357486e+02,-7.440270e+00,0.128374,0.0,1.517502e-01,-3.255327e-06,6.565129e+00,4.090994e-06,17.798812,340.427858,9.973173,-0.082594,0.0,0.195439,-0.000008,1.347465,-3.761121e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250.375,7.386723e-08,-7.918270e-07,9.264627e-07,0.250000,0.0,-2.914799e-09,-4.433896e-16,-4.628935e-08,-1.487982e-15,24.639886,588.556857,41.232957,-0.062935,0.0,0.216423,-0.000006,1.326311,-5.401925e-07
250.376,6.574062e-08,-7.847030e-07,9.254667e-07,0.250000,0.0,-2.861780e-09,-5.272643e-16,-5.120957e-08,-1.409009e-15,24.647498,588.635797,41.230937,-0.062933,0.0,0.216304,-0.000006,1.358494,-5.261711e-07
250.377,5.884593e-08,-7.784109e-07,9.243088e-07,0.250000,0.0,-2.815968e-09,-5.980133e-16,-5.535290e-08,-1.341745e-15,24.653999,588.705717,41.227598,-0.062930,0.0,0.216199,-0.000005,1.389946,-5.132947e-07


In [5]:
dataset[0].load_keypoint_trajectories()


<bound method StressGaitDataset.load_keypoint_trajectories of StressGaitDataset [1 groups/rows]

     participant condition bout
   0       VP_01       omc    1>

In [10]:
import numpy as np